# Using Pandas data frames to analyze single cell electrophysiology data

<img src="http://www.zocalopublicsquare.org/wp-content/uploads/2016/12/Mathews-on-US-China.jpg" width="300" height="300" />

## Day 3 outline

#### Background: 
Today we will analyze patch-clamp data from a single PV neuron in a mouse cortical slice. The experiment we will analyze was performed in current clamp. The experimenter injected 20 different square pulses of current and recorded the voltage response of the neuron to each pulse. Using the data from this experiment, we will create and F-I (frequency-current) curve.

#### 3.0) Import statements

#### 3.1) More on functions
* Review from section 2.1 on day 2: what are functions, why are they useful, how to write them
* More complicated excercise

#### 3.2) For-loops and if-else statements
* An introduction to basic flow control in python using for-loops and if-else statements

#### 3.3) Load single cell electrophysiology data from csv file
* Inspect data using pandas
* Visualize data with matplotlib


#### 3.4) Analyze a single sweep of electrophysiology data
* Calculate the firing rate of the neuron during a single sweep of data

#### 3.5) Create an F-I (frequency - current) curve
* Write a function to calculate firing frequency during one sweep (3.4)
* Employ function in a for-loop to perform the operation for all sweeps

#### 3.6) Extra exercises, if there's time
* Calculate the input resistance of a cell
* Extract spike cutouts
    * Create a phase plot for a spike (rate of change of voltage vs. voltage)
    
#### Appendix
* Additional information about loading binary file formats

## 3.0) Import statements

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal as ss
from __future__ import absolute_import, division, print_function

## 3.1) Functions

Often when you are writing code, you will find that there are certain operations that you want to perform repetitively, only, you want to change the parameters of the operation each time you run it. Functions are great for this. 


Taking the average of a list of numbers is a great, simple example. You can easily write the following line of code:

In [ ]:
my_numbers = [1,4,6,2]
total = my_numbers[0]+my_numbers[1]+my_numbers[2]+my_numbers[3]
average = total/4
print(average)

This works. But it's a pain in the butt. What if we have a new list, but this list has 5 entries instead of 4? Rather than copy and paste all the code above, add my_numbers[4], and change 4 to 5, we can write a function that performs the operation automatically for us.

In [ ]:
def average_nums_in_list(my_l):
    av = sum(my_l)/len(my_l)           # len and sum are built in python functions
    return av

my_new_list = [1,1,3,2,2,6,7,10,3]
print(average_nums_in_list(my_numbers))
print(average_nums_in_list(my_new_list))

Now, every time I want the average of a list of numbers I just have to call "average_nums_in_list" and tell it the list of numbers.

#### Note: 
* Inside this function, we use two other functions (`sum`, which sums the list, and `len`, which gives the length of a list). These are built in python functions. Notice how they help clean up the code in the cell above.


This is a nice example of how functions can be helpful. However, simple things like taking a sum, finding the length of list, and even taking an average (see below and/or numpy documentation on `np.mean()` https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mean.html) are almost always already defined in python or in some other python module (ex: numpy)

In [ ]:
print(np.mean(my_numbers))    # using numpy.mean to compute the average
print(np.mean(my_new_list))

### Exercise
We'll practice writing more of our own functions in a litte bit. For now, let's try using some built in python functions.
* Use numpy.std and numpy.mean to calculate the standard deviation and mean of all columns in the given 2-D matrix ("data_m")
    * https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mean.html
    * https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.std.html

In [ ]:
data_m = np.random.rand(100,10)    # intitalize data matrix
print(data_m.shape)                # 100 rows X 10 columns

In [ ]:
# Compute the mean over the 10 columns and print this array of 10 values
means = 
# Compute the standard deviation over the 10 columns and print this array
stds = 

Bonus: Use matplotlib's bar plot to plot the means and standard deviations for the 10 columns
* https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.bar.html
* Hint: What should 'x' look like if you have 10 columns? - This: https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.arange.html  might help
* Note: Recall from day-2 that seaborn/pandas can really be much better for this sort of thing. This exercise is just here to point out that matplolib exists too, and that there is always more than one way of doing things!

In [ ]:
plt.figure()
plt.bar()      # fill in the correct arguments
plt.show()

* These are all good illustrations of how functions can work, and how we can use online documentation to use predefined function in the way we like. In practice, we often want to write a function that does something a little more complex and stereotyped to our analysis. We'll see more about this later, but for now some examples to think about:
    * perform leak subtraction
    * compute firing rate (more on this later)
    * calcuate input resistance 

## 3.2) for-loops and if-else statements
* for-loops are excellent when you want to iterate over all elements in some sort of data object (list, array, data frame etc.)
* Within these loops, often we only want to perform an operation on the current element when a specific condition is met. For this, we use if-else statements.
* The following examples are meant to walk through the basics of flow control in python by looping over all columns in the pandas data frame defined below:

In [ ]:
data = np.random.rand(100,10)
cols = ['one','two','three','four','five','six','seven','eight','nine','ten']

##convert our data into a data.frame and cacluate indices
df = pd.DataFrame(columns = cols, index = np.arange(0,100,1), data = data)
df.head(5)

* First, we can write a loop that simply goes through all the column names in df and prints them out. Because `cols` is a list, we can *iterate* over it. What this means is that we can take each element of `cols` and do something with it. But how do we access that element? We say that our accessed element is called `c`, and we want to do something with `c`. In this case, we print out what c is.


In [ ]:
cols = df.columns # list of all column names in the df above

for c in cols:
    print(c)

* Another way to achieve this is by iterating over an index. We can do this by using `np.arange`. 

In [ ]:
np.arange(start=0, stop=10, step=1)

In [ ]:
for i in np.arange(start=0, stop=10, step=1):
    print(cols[i])

* Python also lets us combine both of the above examples using enumerate, which gives us an index and the content of that list. This can be really helpful when we want to do calculations over multiple lists in parallel.

In [ ]:
for i, col in enumerate(cols):
    print(i)
    print(col)

Great, we can loop through all the elements of an object (list of column names). Now, let's only perform the print operation for odd numbered columns. We can do this by using the `%` (mod) operator, which gives us the remainder of the operation x / y as an integer. `%` is a great way to do something every other turn.

In [ ]:
for i, col in enumerate(cols):
    if (i+1)%2==0:              # notice that i starts counting from 0, so we must add 1
        pass
    else:
        print(col)

### Exercise

Use np.arange to achieve the same thing as the cell above except this time, don't use an if-else statement.
Hint: think about how you can change step size to achieve the same task

In [ ]:
for     :
    

Now that we understand the basics of flow control and functions, let's try to combine the two to write a slightly more complex function.

### Exercise

Fill in the missing fields of the function below so that it returns the means and standard deviations of either the odd numbered columns, over even numbered columns, of "df", in dictionary format.

* Optional arguments - As you might have noticed when looking through the documentation for numpy.mean and numpy.std, some functions take optional arguments (arguments that have default values but can be changed by the user if desired). Here we'll learn how to write a function this way.
* Dictionaries - Recall your work from section 1.6 on day 1. Dictionary pair keys with a data object e.g. mydict['mean'] could return list of mean values and mydict['std'] could return list of standard deviation values

In [ ]:
def even_or_odd_stats(dataframe, cols='even'):   # note that the default value of columns is even
    
    mydict = {}             # intitalize an empty dictionary to store results
    mydict['mean'] = []     # intialize empty list to store means
    mydict['std'] = []      #intialize empty list to store stds
    
    columns = ___  # define list of columns from the df - what goes here?
    
    if cols=='even':
        # this is where you should loop through all even columns
        for ___ in ___ :
            if :
                mydict['mean'].append()
                mydict['std'].append()
            else:
                pass
            
    elif cols=='odd':
        # this is where you should loop through all odd columns
        for ___ in ___ :
            if :
                mydict['mean'].append()
                mydict['std'].append()
            else:
                pass
            
    return(mydict)     # return results


# ==== Call the function ====
even_or_odd_stats(df,cols='odd')   # can set cols to be 'odd' or 'even'

## 3.3) Load single cell electrophysiology data from .csv file

#### Set file path to day-2 of the python_neurobootcamp
* The 'path' is the list of directories that lead to your file. For example, if I saved something onto the desktop of my computer call "test.py", the path to this file is "/home/charlie/Desktop/" and the filename is "test.py"
    * If you are unsure of the path to your files. You can check by using a file explorer to navigate to the file, right click on it, and view properites. The path should be listed here 
    * The separator (in this case it is a "/") will depend on your operating system: ("/" - Linux/mac, "\" windows) 
* When loading data, it is nice to set one file path that leads to all your data files. Therefore, when you're saving data be conscientious of where you store your files.
* IMPORTANT - Also be conscientious of how you name your files. This can be critical for writing robust, reliable scripts to analyze your data

In [ ]:
pwd  #print working directory

The above command "pwd" prints out what's called the working directory. This is where you "are" right now a.k.a the folder where the file you are working on lives. If your data also lives in the folder (like ours does today) you don't need to specificy the entire path to the files: path="/home/charlie/Desktop/python_bootcamp/python_neurobootcamp/day-3/csv_data/". Instead, you can simply say: path= "csv_data/"

In [ ]:
path = 'csv_data/'

#### The data:

As those of you who do patch clamp know, e-phys data doesn't come nicely packaged into a simple spreadhsheet format. Therefore, we have written a function that converts axon binary files (.abf) into spreadheet format (.csv) for the pruposes of this course. If you are interested in exactly how this process works, look into the file: `binary_file_loading_tools.py` in the `day-3` directory or see Appendix section at the bottom of this notebook. It's a little bit rough, but will help give you an idea of how you can convert binary files to spreadsheets. After completing this course, you should be able to modify this file on your own to suit your needs.

A note on csv files: CSV = Comma-Separated Value file that stores tabular data (numbers and text) in plain text. You can save an excel spreadsheet in .csv and then import it in Python. Let's look at an example. (open up csv file from the day-3/csv_data folder in excel)

#### Load spreadsheet data into pandas
Note the use of index_col=0. Try to get ride of this and see what happens when you load based on default settings. For more information: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

In [ ]:
meta = pd.read_csv(path+"meta_data_PV_3_10_03_2014.csv", index_col=0)
data = pd.read_csv(path+"data_PV_3_10_03_2014.csv",index_col=0)

### Pandas tricks for inspecting data frames
* Some of this might be review from yesterday, but nice to keep in mind

To take a peak at our data, we can type the name of our data frame, followed by ".head()". This will show us the first 5 rows of the data in spreadsheet format. 

In [ ]:
meta.head()

Notice that in our meta data, we have information about the sampling rate (fs), celltype, date of recording, units for ch1 data, and units for ch2 data for each sweep in the binary file that we loaded. This information will become particulalrly useful for analyses where, for example, we want to group anlayses by cell type.

In [ ]:
data.head()

Though the channels aren't labeled, we can determine what each represents by the their units (pico-amps or milli-volts in this case) which we can get above (in meta data).

Notice that data contains the time series for each channel on each sweep. The indexes here represent the time points that each value was acquired at.

Here are a few other tricks that might be useful to refer back to as your analyze your data. (Don't worry too much about the details. This is mostly meant as a reference)

In [ ]:
# What is the shape of the data?
print('Shape of data:')
print(data.shape)

# How many sweeps are there?
print('Rows in meta:')
print(meta.index)

# To only see the number, try:
print('Number of sweeps:')
print(len(meta.index))

# What information do we have about each sweep? i.e. what are the column names?
print('Columns in meta:')
print(meta.columns)

# Can I see only the information for one column?
cols = meta.columns
firstCol = cols[0]
OneCol = meta[firstCol]
OneCol.head()

### Visualization using matplotlib
Before we get into analysis, it's always a good idea to inspect your raw data to make sure you know what's there, and that it was loaded properly.

First let's use Pandas built-in filter function to assign all ch1 traces to a DataFrame called `voltage_traces` and all ch2 traces to a DataFrame called `current_traces`. For basics of filtering, refer back to section 2.9.

Here we use regular expressions to filter the data. These can be very confusing at first, don't worry. But, they are very powerful and worth spending some time getting used to. For more information: https://docs.python.org/2/library/re.html

In [ ]:
voltage_traces = data.filter(regex="ch1.", axis=1)   # if you wanted to filter on rows instead of columns, use axis=0
current_traces = data.filter(regex="ch2.",axis=1)

Plot the data. 
* Note, there's a lot of lines of code in the cell below. Don't worry about trying to memorize it all. A lot of plotting will be figuring it out once, then copying and pasting the same code over an over. You'll get a chance to practice this in a moment.

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True)
ax[0].plot(voltage_traces)
ax[0].legend(voltage_traces,loc='upper right')
ax[0].set_title('Voltage traces')
ax[0].set_ylabel(meta['ch1_units'][0])
ax[0].set_xlabel('Time (s)')

ax[1].plot(current_traces)
ax[1].legend(current_traces,loc='upper right')
ax[1].set_title('Current traces')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel(meta['ch2_units'][0])

plt.tight_layout()
plt.show()

This is pretty messy. Can we make it better?

### Exercise: 

* Try plotting only the last sweep of both current and voltage so that we can see what's going on more easily.
* Use the code from above as a starting point for how to create your own plot. Ask a TA or instructor if you need help.

First, let's find the column id for the last sweep using the "tricks" above (there are multiple ways to do this).
* In python, we can use "-1" to grab the last index of something. This is equivalent to using "end" in matlab.

In [ ]:
lastSweep_voltage = voltage_traces.columns[-1]     
lastSweep_current = current_traces.columns[-1]

print(lastSweep_voltage)
print(lastSweep_current)

Now, use these  column id's to select only the columns containing data for the last sweep:

In [ ]:
last_voltage = voltage_traces[lastSweep_voltage]
last_current = current_traces[lastSweep_current]

* Use these column id's to only plot the traces for these sweeps:

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True)
ax[0].plot(last_voltage)
ax[0].set_ylabel(meta['ch1_units'][0])
ax[0].legend([lastSweep_voltage], loc='upper right')
ax[1].plot(last_current)
ax[1].set_ylabel(meta['ch2_units'][0])
ax[1].set_xlabel('Time (s)')
ax[1].legend([lastSweep_current], loc='upper right')
plt.show()

That looks better, but can we zoom in on some of the spikes in the voltage traces?

### Exercise
* Plot only time points from 0.02 to 0.10 seconds:

In [ ]:
# define the range if indexes you want to select
inds = data.index[(0.02<data.index) & (data.index < 0.1)]

In [ ]:
# Now repeat the plot above, only for the given indeces
fig, ax = plt.subplots(2,1,sharex=True)
ax[0].plot(last_voltage[inds])
ax[0].set_ylabel(meta['ch1_units'][0])
ax[0].legend([lastSweep_voltage], loc='upper right')
ax[1].plot(last_current[inds])
ax[1].set_ylabel(meta['ch2_units'][0])
ax[1].set_xlabel('Time (s)')
ax[1].legend([lastSweep_current], loc='upper right')
plt.show()

## 3.4) Analyze a single sweep of electrophysiology data

A common way to show a neuron's response to input is to make a frequency-intensity curve (F-I curve).  The frequency of firing is plotted on the y-axis and the amplitude of the current injection that caused the spikes is plotted on the x-axis.  

From looking at these F-I curves you can identify if the cell is transiently firing, whether the spike rate reaches a maximum at some high current step, the gain of firing (the slope of the line, in Hz/pA), etc.  To construct a F-I curve we need to first find the number of spikes per unit time in each sweep and the current injection that drove those spikes. Then plot it.

First, we will perform the analysis for just one sweep of data. Then we will define a function to automate the process, run the function on all sweeps using a for-loop, and plot the results in a F-I curve.  

Becasue we have already pulled out the last voltage trace, let's go ahead and analyze this time-series first.

The analysis will consist of a few key steps:
* Step 1: Locate the time points of all spikes
* Step 2: Find the time window and magnitude of current injection
* Step 3: Calculate the firing rate

#### Step 1: Locate the time points of all spikes
* There are a couple of ways to do this. We will do it by locating the relative maxima of the voltage trace during the current injection
* We will also set a threshold above which to detect maxima (to avoid counting local maxima in the baseline (noise) as a spikes)

In [ ]:
threshold = -20  
last_voltage_thresholded = last_voltage[last_voltage>threshold]  

Let's take a look at what happened when we thresholded the voltage trace:

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(last_voltage_thresholded)
ax[0].set_xlabel('Time (s)')
ax[0].set_title('Thresholded trace')
ax[1].plot(last_voltage)
ax[1].set_xlabel('Time (s)')
ax[1].set_title('Raw trace')
plt.tight_layout()
plt.show()

Using a a threshold of -20, we have selected all the voltage points above -20 mV. In other words, these are all points during which an action potential was happening.

*Scipy*, another useful Python package, has a function that finds the indices where relative extrema occur: `argrelextrema`
This function takes two required arguments: data and comparator (for details see [documentation](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.signal.argrelextrema.html)).

* Note: `last_voltage_thresholded` is a Pandas DataFrame. `argrelextrema` expects data to be an Array, not a DataFrame.
* Note: `argrelextrema` also takes an optional argument: "order" that specifies the amount of points on each size to use for the comparison.

In [ ]:
indexes_of_maxima = ss.argrelextrema(data=last_voltage_thresholded.values, comparator=np.greater, order=1)

The Scipy function returns indexes i.e. 1,2 3... BUT we want the time points (which are stored in the index of our DataFrame)

In [ ]:
spike_times = last_voltage_thresholded.index[indexes_of_maxima] 

It's possible that the voltage trace is a little noisy, even during a spike. This might cause a spike to be double counted by the Scipy function, which is only locating relative/local maxima. To fix these cases, we impose a constraint based on the refractory period of neuron.

In [ ]:
refractory_period_low_limit = 0.002  # 2ms is a lower limit on the refracotry period for a neuron

Now, let's calculate the interspike intervals for all the maxima we located (using numpy's ediff1d function) and exclude any spikes that violate the refractory period.
* np.ediff1d: Calculates the difference between all adjancent elements in a 1d array: https://docs.scipy.org/doc/numpy/reference/generated/numpy.ediff1d.html
* np.arghwere: Locates the indexes where the given condition is met: https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.argwhere.html

In [ ]:
interspike_interval = np.ediff1d(spike_times)
VIOLATION_indexes = np.argwhere(interspike_interval<refractory_period_low_limit)
spike_times = spike_times.delete(VIOLATION_indexes)
print(spike_times)

We now have an array of time points where we think spikes occured. Let's plot them to see if we were successful

In [ ]:
plt.figure()
plt.plot(last_voltage)
plt.plot(spike_times, last_voltage[spike_times], ".", markersize=20) # plot a point at each time point where a spike was detected
plt.show()

This is looking pretty good. 

### Exercise
What happens if you now change the threshold to -60 and re-run the cells above? This should help illustrate why it's important to choose the apporpriate threshold when detecting spikes.

Now, let's zoom in on some spikes to make sure we really are counting each individual spike

In [ ]:
plt.figure()
plt.plot(last_voltage[inds])
plt.plot(last_voltage[inds][spike_times],".",markersize=9)
plt.show()

Great, looks like we are capturing all the spikes, not double counting, and not counting any noise as spikes. 

#### Step 2: Find out the time window of current injection
This is pretty straightforward for this experiment because as we saw earlier, it's just square pulse injections. Therefore, we can just find the time points where current == max(current).

In [ ]:
current_magnitude = max(last_current.values)  
current_inj_times = last_current.index[last_current==current_magnitude] # time points when current is being injected   
current_inj_length = max(current_inj_times) - min(current_inj_times) # duration in seconds
print("Current injection: "+str(current_magnitude)+" pA")

Before we calculate the firing frequecny, there is one last check we must perform. It is possile that some spontaneous spikes occured outside the current injection window (it didn't happen in this case, but it could and that could change our results). To handle this, we remove all spikes times outside of the current injection window. To do this, we create a boolean mask (list of True and False) which we can then use to only grab indixes where there is a one:

In [ ]:
boolean_array = (max(current_inj_times)>= spike_times) & (spike_times >= min(current_inj_times)) # boolean array (0's and 1's)
print(boolean_array)

Now, use the boolean mask to get only the spike times inside the appropriate time window

In [ ]:
spike_times = spike_times[boolean_array]   # only saving spike times where boolean_array=True e.g. during current injection

#### Step 3: Calculate the firing rate 
There are couple of different ways to calculate firing frequency. They are both useful and tell you different things.  
The most straighforward way is to simply count the number of spikes and divide by the amout of time (in seconds) that the current was being injected.

In [ ]:
n_spikes = len(spike_times)
spike_freq = n_spikes/current_inj_length

print("spiking frequency = " + str(round(spike_freq,2))+" Hz")

The second way to go about this is to calculate the average interspike interval. This is particularly useful if the neuron does not continue to fire or adapts during the current injection.

In [ ]:
ISI = np.ediff1d(spike_times) #interspike interval
meanISI = np.mean(ISI)  # calculate the mean ISI
spike_freq = 1/meanISI  # ISI is in units of seconds, divide 1s/meanISI to get mean firing frequency
print("spiking frequency = " +str(round(spike_freq,2))+"Hz")

For this cell, both methods give us basially the same result because as we saw above, this neuron fires pretty regularly during the entire stimulation. This isn't necesarily the case, so it's good to check. 

One final check we can perform is just to look at all the raw ISI values to make sure there wasn't a change in frequency over the course of the stimulation (high firing at the beginnning, slower at the end)

In [ ]:
plt.figure()
plt.hist(ISI*1000,bins=50) # convert to ms by multiplying by 1000
plt.ylabel('Count')
plt.xlabel('Interspike interval (ms)')
plt.show()

This looks pretty good with the exception of a few outliers. It also confirms that there are no refractory period violations. 


### Exercise
Go back and re-run everything from "Step 1" but this time, DO NOT run the cells that corrected for refractory period violations. This should illustrate why this final histogram can be a good check on your analysis.

####  Summary of single sweep analysis 
Although what we really want is the F-I curve for the cell, analyzing one sweep at a time, like we did here, is often a good idea. This way, you can make sure that the code you write is really analyzing the data in the way you intend.

## 3.5) Create an F-I curve for a single cell

* To do this, we will perform the above analysis many times. Once for each current injection/sweep
* Because we are doing the same thing many times, rather than write all of the above code 20 times (once for each sweep), we'll write a "for" loop that performs the same operation many times
* When you are performing an operation many times, it is often nice to write function to perform this operation. We will write a function that returns the firing frequency for any sweep

#### Write a function that returns the firing rate for a single sweep of data

In [ ]:
def get_firing_rate(v, threshold, tstart, tend, method="ISI"):
    '''
    ================ This is called a doc string ====================
    ========== It tells you how to use the function =================
    ======= It is what is printed when you call "help" ==============
    Arguments:
        v (data frame): array of voltage values during one sweep with the index being the time series
        threshold (float): cut-off to count spikes
        tstart (float): time current injection begins
        tend (float): time current injection ends
        method (string, optional): method for calculating the firing rate
                ISI: use interspike interval
                AVG: use average over whole current injection window
    Output:
        firing_rate (float): firing rate during the period defined by tstart and tend
    '''
    refractory_limit=0.002 # set refractory period limit (to avoid double counting spikes)
    current_duration = tend-tstart
    
    v_thresh = v[v>threshold]
    
    spike_indexes = ss.argrelextrema(v_thresh.values, np.greater,order=1)  # indices of spikes (0,1,2...)
    spike_times = v_thresh.index[spike_indexes]  # convert to time (in seconds)   
    
    # get rid of refractory violations
    interspike_interval = np.ediff1d(spike_times)
    VIOLATION_indexes = np.argwhere(interspike_interval<refractory_limit)
    spike_times = spike_times.delete(VIOLATION_indexes)    # delete any spike times that occured too soon after a previous spike
    
    # get rid of spikes outside the current injection window
    tf = (tend >= spike_times) & (spike_times >= tstart) # boolean array (0's and 1's)
    spike_times = spike_times[tf]   # only saving spike times where tf==1 e.g. during current injection
    
    if method=="ISI":
        # calculate the firing frequency using interspike interval
        # in case there were no spikes, set firing rate to 0 Hz
        if spike_times.size==0:
            firing_rate=0  # there were no spikes, so firing rate is 0
        else:
            ISI = np.mean(np.ediff1d(spike_times))
            firing_rate = 1/ISI
    elif method=="AVG":
        firing_rate = len(spike_times)/current_duration
    
    return firing_rate

There is alot going on in that function. If you are confused on any part of it, please ask the instructor or TAs for assitance. 
* You should understand how the function works, and how to modify it if you need/want to
* You should understand the difference between required and optional function arguments
* You should understand the if-else statements at the end of the function definition
    * Note: check out what happens when you call "help" on the function we just wrote:

In [ ]:
help(get_firing_rate)

#### Write a for-loop to calculate the firing rate for every sweep of data
* First intitalize the constants that we will use throughout the analysis
    * threshold
    * current_duration

In [ ]:
# set threshold above which to detect spikes
threshold = -20   

# Find current duration (same for all sweeps so we can just do this once)
current_magnitude = max(last_current.values)  
current_inj_times = last_current.index[last_current==current_magnitude]  # time points when current is being injected  
I_start = min(current_inj_times)
I_end = max(current_inj_times)
current_duration = max(current_inj_times) - min(current_inj_times)      # duration in seconds   

Loop over all the sweeps, find the current magnitude and firing rate, print them out

Define the list of sweeps to loop over:

In [ ]:
sweeps = meta.index

In [ ]:
for i, sweep in enumerate(sweeps):
    
    # Call our function that we wrote above (get_firing_rate)    
    firing_rate = get_firing_rate(data['ch1_'+sweep], threshold, I_start, I_end, method="ISI")  
    
    # get current injection magnitude
    # Note how we deal with negative current injections using if-else statements
    I_mag = max(data['ch2_'+sweep])
    if I_mag == 0 and min(data['ch2_'+sweep])==0:
        I_mag = 0
    elif I_mag == 0 and min(data['ch2_'+sweep]) != 0:
        I_mag = min(data['ch2_'+sweep])
    
    # print the result
    print(sweep+":  "+"I: "+str(I_mag)+" "+meta['ch2_units'][i]+ ",   Firing rate: "+str(firing_rate)+" Hz")

There's a lot of stuff going on here as well. Some key takeaways:
* Make sure you understand the for loop, what it's doing, and why it's useful
* Make sure that the if-else statements inside the for loop are making sense
* If you are confused on these points, ask the instructor or a TA

Now that we understand how the code above is working, let's run it again but this time actually save the results instead of just printing them out. To save our results, let's create a new data frame called "results" to store everything

In [ ]:
results = pd.DataFrame(columns=["Current Injection", "Firing Rate"],index = meta.index)
results.head()

We've now created an empty data frame to store the results for all data sweeps. Using a for loop, let's again calculate the current injection and firing rate for each sweep and fill up the empty results data frame

In [ ]:
for i, sweep in enumerate(sweeps):
    firing_rate = get_firing_rate(data['ch1_'+sweep],threshold,I_start,I_end,method="ISI")
    
    I_mag = max(data['ch2_'+sweep])
    if I_mag == 0 and min(data['ch2_'+sweep])==0:
        I_mag = 0
    elif I_mag == 0 and min(data['ch2_'+sweep]) != 0:
        I_mag = min(data['ch2_'+sweep])
       
    results['Current Injection'][sweep]=I_mag
    results['Firing Rate'][sweep]=firing_rate

Now, we have stored all our results into a new data frame.

In [ ]:
results

Now, we can create our f-i plot for this neuron

In [ ]:
plt.figure()
plt.plot(results['Current Injection'].values, results['Firing Rate'].values, ".-", color="r", lw=2,
         markersize=5, alpha=0.6)
plt.xlabel('Current '+meta['ch2_units'][0])
plt.ylabel('Firing rate (Hz)')
plt.title('F-I curve')
plt.show()

Cool, we now have an F-I curve for this PV neuron. 

### Excercise: do the same analysis but calculate frequency as n_spikes/time instead of using the ISI as we just did
* Add the result to the data frame called results and name this column "Firing rate (avg)"
* hint: one way to add a column to a data frame is to make a series and then add it to the original data frame
    * newcolumn = pd.series(index=, name=)
    * pd.concat((results,newcolumn),axis=1)
* Once you've accomplished all this, plot the results alongside the trace above

#### Create  new column using pandas series 
* https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html

In [ ]:
newColumn = pd.Series()
# answer: newColumn = pd.Series(index=sweeps, name="Firing Rate (avg)")

#### Loop over all the data and fill in the new column

In [ ]:
for :
    
    # get firing rate
    
    # add to new column
    
    

'''
answer:
for i, sweep in enumerate(sweeps):
    
    firing_rate = get_firing_rate(data['ch1_'+sweep],threshold,I_start,I_end, method="AVG")
    
    newColumn[sweep]=firing_rate
'''

#### Concatenate the newColumn with results
* https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html

In [ ]:
results = pd.concat()   # concatenate
results.head()          # look at results

'''
results = pd.concat((results,newColumn),axis=1)
results.head()
'''

Plot the results for both methods on the same figure

In [ ]:
# Use examples of plotting from above




'''
plt.figure()
plt.plot(results['Current Injection'].values, results['Firing Rate'].values, ".-", color="r", lw=2,
         markersize=5, alpha=0.6)
plt.plot(results['Current Injection'].values, results['Firing Rate (avg)'].values, ".-", color="b", lw=2,
         markersize=5, alpha=0.6)
plt.xlabel('Current '+meta['ch2_units'][0])
plt.ylabel('Firing rate (Hz)')
plt.legend(['ISI', 'avg'])
plt.title('F-I curve')
plt.show()
'''

Basically, we get the same result. However, we do see a clear difference for the first positive current step, when the cell did not continue to fire over the course of the current injection but only fired a burst of spikes at the beginning of stimulation

## 3.6) Extra exercises, if there's time

#### Calculate the input resistance of the cell (Easier)
Recall ohms law: voltage = current * resistance (V=IR). We know the current and we know the voltage so we can solve for R, the cell's input resistance. Using ohms law and the hints below, solve for this cell's input resistance.

* Hint 1: Input resistance is a passive property of a neuron, therefore we'll have to measure it when voltage activated channels are not opening/closing. Remember from our analysis above, there were a bunch of negative current steps that elicited no action potentials.

* Hint 2: Even though the negative current steps didn't cause action potentials to fire there could still have been some channels opened, changing the resistance of the cell (for example, Ih currents). Therefore, it's probably best to use voltage measurements when the cell was in a steady state (later on in the current injection).

* Hint 3: Ohms law is a linear relationship. In other words, if we plot voltage vs. current we should see a straight line (if no channels were opening and closing). The slope of this line is R. Plot voltage vs. current for all negative current injections and meausre the slope at the point where the line is most linear.

In [ ]:
# ==============================   Solution    ========================================

negative_current_sweeps = results.index[[i for i in results['Current Injection']<0]]
tstart=0.4
tend=0.5
steady_state_voltage = []
current=[]
for sweep in negative_current_sweeps:
    steady_state_voltage.append(np.mean(data['ch1_'+sweep][tstart:tend]))
    current.append(results['Current Injection'][sweep])

plt.figure()
plt.plot(current, steady_state_voltage,'.-')
plt.ylabel('mV')
plt.xlabel('pA')


# remove last two current injections because seems to get non-linear there
# and convert to Volts and Amps
R = ((steady_state_voltage[-2]-steady_state_voltage[0])/1000)/((current[-2] - current[0])/10e12)
R_mohms = round(R/10e6,2)
plt.title('Input resistance = ' + str(R_mohms)+ ' Mohms')

plt.show()

#### Create a phase-plot for this cell (More difficult)
A phase plot is a plot of voltage vs. the derivative of the voltage (rate of change) during an action potential. Here's an example:
<img src = 'phase_plot.png'>

Your goal is to create a phase plot for the action potentials fired during the 500pA current injection. There a few steps involved to do this:
* Step 1: Generate and array of "spike cutouts" during this current step. In other words, make a 2D numpy array where the rows are time points and the columns are individual action potentials. Do this by taking 1ms of data before an AP peak and 2ms of data after a peak. Hint: remember how we found the time points of AP peaks earlier...

* Step 2: Generate an equivalent numpy array except in this array, each column is the derivative of the action potential in the array from step 1

* Step 3: Take the mean of all columns in both arrays and then plot action potential mean on the x-axis vs. the derivative mean on the y-axis

In [ ]:
# ============================== Solution =======================================

# get the voltage values for the 500pA sweep (last sweep)

voltages = voltage_traces[voltage_traces.columns[-1]]

# find spike times
threshold=-20
voltages_thresh = voltages[voltages>-20]
indexes_of_maxima = ss.argrelextrema(data=voltages_thresh.values, comparator=np.greater, order=1)
spike_times = voltages_thresh.index[indexes_of_maxima]


# get rid of refractory violations
interspike_interval = np.ediff1d(spike_times)
VIOLATION_indexes = np.argwhere(interspike_interval<0.002)
spike_times = spike_times.delete(VIOLATION_indexes)



# get spike cutouts
n_spikes = len(spike_times)
len_spike_window = len(voltages[0:0.003])

# make an empty arrays to hold spike cutouts and derivatives
spike_cutouts = np.empty((n_spikes,len_spike_window-1))
derivative = np.empty((n_spikes, len_spike_window-2))

# fill arrays
fs = meta['fs'][0]
for i in range(0, n_spikes):
    spike_cutouts[i,:] = voltages[(spike_times[i]-0.001):(spike_times[i]+0.002)].squeeze()
    derivative[i,:] = np.ediff1d(spike_cutouts[i,:])/(1/fs)

spike = np.mean(spike_cutouts,axis=0)[:-1]
deriv = np.mean(derivative,axis=0)

plt.figure()
time = np.linspace(0,0.003,len(spike))
plt.subplot(221)
plt.title('Action potential')
plt.plot(time,spike)
plt.xlabel('Time (s)')
plt.ylabel('mV')

plt.subplot(223)
plt.title('Derivative')
plt.xlabel('Time (s)')
plt.ylabel('mV/s')
plt.plot(time,deriv)

plt.subplot(122)
plt.title('Phase plot')
plt.ylabel('dV/dt (mV/s)')
plt.xlabel('mV')
plt.plot(spike, deriv,'.-')
plt.tight_layout()

plt.show()

## Appendix

#### Loading binary files (.abf, .dat, etc.) using the python package, neo
 neo is an open source python package, like numpy or pandas. It was written specifically to help load binary file formats from electrophysiology data into python. We have written a function that uses this package to load .abf files into a .csv data format (spreadsheet). If you are interested in learning more about this, ask one of your TA's or intsructors, or just go look at the file binary_file_loading_tools.py that is located in the day 3 folder. Note, in order to run the loading function, you will need neo installed on your machine. To do this, type: "pip install neo" into a terminal window on your machine. For more information on neo, see: http://neuralensemble.org/neo/
 